In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [59]:
cd drive/Houston/

[Errno 2] No such file or directory: 'drive/Houston/'
/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
hsi = []
for i in range(3):
  img_path = "pca/hsi_pc" + str(i) + ".png"
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  hsi.append(img)
        
hsi = np.array(hsi)

lidar = cv2.imread("pca/lidar.png", cv2.IMREAD_GRAYSCALE)
lidar = lidar.reshape(1, lidar.shape[0], lidar.shape[1])


In [62]:
print(hsi.shape)
print(lidar.shape)

(3, 349, 1905)
(1, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
hsi_train_data = []
hsi_test_data = []
lidar_train_data = []
lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            hsi_train_data.append(hsi[:, i, j])
            lidar_train_data.append(lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            hsi_test_data.append(hsi[:, i, j])
            lidar_test_data.append(lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
hsi_train_data = np.array(hsi_train_data)
lidar_train_data = np.array(lidar_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [67]:
print(hsi_train_data.shape)
print(lidar_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 3)
(2832, 1)
(2832,)
(12197,)


In [0]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

In [69]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [71]:
padded_hsi = np.lib.pad(hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_lidar = np.lib.pad(lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')
print(padded_hsi.shape)
print(padded_lidar.shape)

(3, 375, 1931)
(1, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
hsi_train_patches = []
hsi_test_patches = []
lidar_train_patches = []
lidar_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      hsi_train_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_train_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      hsi_test_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_test_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [74]:
hsi_train_patches = np.array(hsi_train_patches)
hsi_test_patches = np.array(hsi_test_patches)
lidar_train_patches = np.array(lidar_train_patches)
lidar_test_patches = np.array(lidar_test_patches)
print(hsi_train_patches.shape)
print(hsi_test_patches.shape)
print(lidar_train_patches.shape)
print(lidar_test_patches.shape)

(2832, 27, 27, 3)
(12197, 27, 27, 3)
(2832, 41, 41, 1)
(12197, 41, 41, 1)


In [75]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

hsi_model = Sequential()
hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


hsi_model.add(Flatten())
hsi_model.add(Dense(FC1, activation='relu'))
hsi_model.add(Dropout(0.75))
hsi_model.add(Dense(FC2, activation='relu'))
hsi_model.add(Dropout(0.6))
hsi_model.add(Dense(NUM_CLS, activation='softmax'))


hsi_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 500)       54500     
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 500)       2000      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 500)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 100)         1250100   
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 100)         400       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
__________

In [76]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

lidar_model = Sequential()
lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


lidar_model.add(Flatten())
lidar_model.add(Dense(FC1, activation='relu'))
lidar_model.add(Dropout(0.75))
lidar_model.add(Dense(FC2, activation='relu'))
lidar_model.add(Dropout(0.6))
lidar_model.add(Dense(NUM_CLS, activation='softmax'))


lidar_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 21, 21, 500)       18500     
_________________________________________________________________
batch_normalization_6 (Batch (None, 21, 21, 500)       2000      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 500)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 5, 5, 100)         1250100   
_________________________________________________________________
batch_normalization_7 (Batch (None, 5, 5, 100)         400       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
__________

In [0]:
hsi_model.load_weights('Models/hsi_model_weights.h5')
lidar_model.load_weights('Models/lidar_model_weights.h5')

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [79]:
test_cls = test_labels - 1
prediction = hsi_model.predict(hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.6285152086578667
Average Accuracy of training samples :  0.6665988960020399
Kappa statistics of training samples :  0.5958508339430351
Each class accuracy of training samples :  [0.8247639  0.66791745 0.5907781  0.91914191 0.74119339 0.93442623
 0.75595238 0.5035461  0.75259679 0.62673392 0.35395932 0.50643087
 0.54761905 0.62457338 0.64935065]
Confusion matrix : [[ 786   95    0   63    0    0    0    0    0    0    3    0    0    0
     6]
 [  64  712    0  209    0    0    4    0    1    0    0    0    0    5
    71]
 [   2    0  205   33    0    5    7   11    0   21    1   18    1    2
    41]
 [   3   46    0  557    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    3 1031    0    0   49   47    1    0  176    0   56
    28]
 [   2    0    0    0    0  114    0    0    1    0    0    0    0    0
     5]
 [  11   39    3   73    0   19  889   21   64    0   36    0   21    0
     0]
 [   0    1    3   19    4    0 

In [0]:
test_cls = test_labels - 1
prediction = lidar_model.predict(lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.5034844633926375
Average Accuracy of training samples :  0.5351069735432643
Kappa statistics of training samples :  0.4611027347129306
Each class accuracy of training samples :  [0.38403042 0.37081851 0.57446809 0.78197065 0.52842105 0.55665025
 0.50909091 0.61577181 0.38831615 0.79407407 0.82731707 0.27564506
 0.24725275 0.81533101 0.35744681]
Confusion matrix : [[101   9   0   0  29  16   1   0  39  41   2   0   4   2  19]
 [177 521  44  39  66   1  12  26  80  79  42 140  42   7 129]
 [  7   0 216   0   0   0   0   0   1   0   0   0   0   0 152]
 [ 11   0   0 373   3   0  64  12   6   8   0   0   0   0   0]
 [ 88  42   0   0 251   0   0   0  17  33   3  20  20   0   1]
 [ 17   4   0   7  38 113   0   0   1  14   0   7   2   0   0]
 [ 20  10   0 448  34   4 840 107  94  40  43  10   0   0   0]
 [  9   3   0 106   5   0  34 734   0   8   0 292   0   0   1]
 [ 49 210   0  42 439   7  89   2 678  47  65  61  56   0   1]
 [ 15  16   0   7   1   0

In [0]:
from tensorflow.python.keras.models import Model
intermediate_layer_hsi_model = Model(hsi_model.layers[0].input,outputs=hsi_model.layers[6].output)
intermediate_layer_lidar_model = Model(lidar_model.layers[0].input,outputs=lidar_model.layers[6].output)

hsi_train_flatten = intermediate_layer_hsi_model.predict(hsi_train_patches)
lidar_train_flatten = intermediate_layer_lidar_model.predict(lidar_train_patches)

hsi_test_flatten = intermediate_layer_hsi_model.predict(hsi_test_patches)
lidar_test_flatten = intermediate_layer_lidar_model.predict(lidar_test_patches)

In [0]:
print(hsi_train_flatten.shape)
print(lidar_train_flatten.shape)
print(hsi_test_flatten.shape)
print(lidar_test_flatten.shape)

(2832, 400)
(2832, 400)
(12197, 400)
(12197, 400)


In [0]:
train_fusion = np.concatenate((hsi_train_flatten, lidar_train_flatten), axis=1)
test_fusion = np.concatenate((hsi_test_flatten, lidar_test_flatten), axis=1)
print(train_fusion.shape)
print(test_fusion.shape)

(2832, 800)
(12197, 800)


In [0]:
from tensorflow.keras.layers import concatenate
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD


fusion_model = Sequential()
fusion_model.add(Dense(512, input_dim=train_fusion.shape[1], activation='relu'))

fusion_model.add(Dropout(0.8))
fusion_model.add(Dense(256, activation='relu'))
fusion_model.add(Dropout(0.7))
fusion_model.add(Dense(128, activation='relu'))
fusion_model.add(Dropout(0.6))
fusion_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE/2, decay=1e-6, momentum=0.9, nesterov=True)
fusion_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

fusion_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               410112    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 15)                1935      
Total para

In [0]:
history = fusion_model.fit(train_fusion, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 1s 300us/step - loss: 0.0458 - acc: 0.9901
Epoch 2/50
2832/2832 [==============================] - 1s 318us/step - loss: 0.0298 - acc: 0.9944
Epoch 3/50
2832/2832 [==============================] - 1s 308us/step - loss: 0.0336 - acc: 0.9944
Epoch 4/50
2832/2832 [==============================] - 1s 286us/step - loss: 0.0326 - acc: 0.9929
Epoch 5/50
2832/2832 [==============================] - 1s 281us/step - loss: 0.0303 - acc: 0.9922
Epoch 6/50
2832/2832 [==============================] - 1s 283us/step - loss: 0.0310 - acc: 0.9929
Epoch 7/50
2832/2832 [==============================] - 1s 302us/step - loss: 0.0336 - acc: 0.9936
Epoch 8/50
2832/2832 [==============================] - 1s 312us/step - loss: 0.0374 - acc: 0.9933
Epoch 9/50
2832/2832 [==============================] - 1s 300us/step - loss: 0.0400 - acc: 0.9912
Epoch 10/50
2832/2832 [==============================] - 1s 271us/step - loss: 0.0323 - acc: 0.9929
Epoch 11/

In [0]:
fusion_model.load_weights('Models/fusion_model_weights.h5')

In [0]:
test_cls = test_labels - 1
prediction = fusion_model.predict(test_fusion).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.6996802492416168
Average Accuracy of training samples :  0.720504257626817
Kappa statistics of training samples :  0.6736722373336643
Each class accuracy of training samples :  [0.84640172 0.67872807 0.76617954 0.81715363 0.84105263 0.83453237
 0.76648841 0.56923077 0.61443932 0.91559371 0.70958512 0.40499307
 0.66386555 0.73076923 0.64855072]
Confusion matrix : [[788 133   0   6   0   0   0   0   1   0   3   0   0   0   0]
 [ 43 619   9  60  35   3  12  14  41  10   0  20   0   0  46]
 [  0   0 367   0   0   0   0   0   6   0   3   0   0   0 103]
 [ 23   6   0 867   1   0  80  13   4  67   0   0   0   0   0]
 [  4  20   0  12 799   0   0   0  37  24   1  47   6   0   0]
 [ 14   0   0   1   0 116   0   3   0   0   0   4   1   0   0]
 [  0  12   1  79  16   6 860  37  86   1   1   0   9   0  14]
 [  0   1   0  13  12   0  12 518   0   0   0 280   0   0  74]
 [ 25  86   1  18 158  18  74   1 800  30  22  32  30   0   7]
 [  0   0   1   0  22   0 

In [0]:
fusion_model.save_weights('Models/fusion_model_weights.h5')